# Segment each fibre for 3D volume with trained UnetID
- Author: Rui Guo (KU Leuven), rui.guo1@kuleuven.be
- Date: July 2022

## Import packages

In [ ]:
import fibresegt as fs
import numpy as np
import matplotlib.pyplot as plt

## Path for output and pretained model

In [ ]:
output_dir = './output/demo/'
trainedNet_dir = fs.join(output_dir, 'checkpoint/')

## Load data

### In case your data are saved as images

In [ ]:
# dataset_folder_img = './data/demo/data_group/images/'
# orig_3Ddata, ids_3Ddata = fs.load_3d_dataset(dataset_folder_img)

### In case your data are saved as h5 files

In [ ]:
dataset_folder = './data/demo/data_group/h5file/'
orig_3Ddata, ids_3Ddata = fs.load_hdf5_data(dataset_folder, 
                                            dtype=np.uint8) 

## Visulize data
- **set_id:** Specify which sets of data you want to see
- **visulize_plane_range:** Specify which plane and plane size of the data you want to see
- **overlay:** Set True if you want to see the overlay results

In [ ]:
data_list = [orig_3Ddata]
set_id = ids_3Ddata[0]
slice_range, height_range, width_range = [0], [0, 1000],[0, 1000]
# slice_range, height_range, width_range = [0, 200], [0],[0, 200]
# slice_range, height_range, width_range = [0, 200], [0, 200], [0]

visulize_plane_range = [slice_range, height_range, width_range]
overlay = False
show_model=None

In [ ]:
fs.visualize_3d_data(data_list, set_id, visulize_plane_range, overlay=False, show_model=None)

## Automatically segment fibre

- **1. checkpoint_id**  
This is used to which trained model will be used. The default is the last one. You can also specify the specific number
- **2. crop_input_shape**   
You can simple choose this according to your training image size
- **3. batch_size**   
The size is used to segment multiple slices at the same time.
- **4. save_image_num**   
You can set 10, then only 10 images are saved. The default value is 'full', which means all data will be saved.
- **5. save_info**   
Only the inner fibre are saved by setting it as 'inner', or 'default', then all information (inner, edge, whole fibre) are saved. 
- **6. save_orig_results**  
If you set this variable as True, then the results before prost processing are saved.  
In order to save the space, you can set it as False. Then only the results after post processing will be saved.  
- **7. hardware_acltr**  
You can also use CPU to segment your data. Default is GPU will be used.
- **8. postproc_param**  
If you don't want to use post processing, you can set it as None.  
This post processing is used to remove the noise by using opening operation. So you don't need to change the method.  
The most things you should be careful is the 'kernel', which is morphological structuring element.  For post process, it is important to choose an appropriate parameter for kernel size.  
The default is "kernel": {"kernel_shape":"disk", "kernel_radius":4}, you can set the kernel radius as 1, 2, 3, 4, 5 according your requirements.  
Of course, you can also choose the ellipse kernel shape, now you should input the kernel size instand of kernel radius, such as: {"kernel_shape":"ellipse", "kernel_size":4}. The size you can choose any value as your requirements.

In [ ]:
dataset            = orig_3Ddata
net_var            = 'UnetID'
output_dir         = output_dir
trainedNet_dir     = trainedNet_dir
dataset_name       = 'Default' # Default is segm_results_3D
checkpoint_id      = 'last_id' # or checkpoint_id=200
crop_slice_shape   = (64,64,1)
batch_size         = 500 # You can set it small if your image size is large.
save_format        = ['H5', 'png']
save_image_num     = 'Full' 
save_info          = 'default' 
save_orig_results  = False
hardware_acltr     = 'GPU' # Default is 'GPU'
load_checkpoint    = True
postproc_param     =  {"method": "open", "kernel": {"kernel_shape":"disk", "kernel_radius":4}, 
                       "iteration": 1, "save_postproc_results":True} # Remove small artifacts

In [ ]:
fs.apis.segm_3d_data(dataset=dataset, 
                    net_var=net_var, 
                    output_dir=output_dir,
                    trainedNet_dir=trainedNet_dir,
                    dataset_name=dataset_name,
                    checkpoint_id=checkpoint_id, 
                    crop_slice_shape=crop_slice_shape,
                    batch_size=batch_size,
                    save_format=save_format,
                    save_image_num=save_image_num,
                    save_info=save_info, 
                    save_orig_results=save_orig_results,
                    hardware_acltr=hardware_acltr,
                    load_checkpoint=load_checkpoint,
                    **postproc_param)